# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: I used **LoRA** as my PEFT technique. LoRA is the only PEFT technique that is compatible with all models at this time.
* Model: I used **GPT-2** as my model. This is a relatively small model that is compatible with sequence classification and **LoRA**.
* Evaluation approach: The evaluation approach covered in this project was the `evaluate` method with a **Hugging Face** `Trainer`.
* Fine-tuning dataset: I use a dataset from Hugging Face's datasets library [stanfordnlp/imdb](https://huggingface.co/datasets/stanfordnlp/imdb).

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

### Import libraries

In [13]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer

### Loading the model

In [14]:
dataset = load_dataset("stanfordnlp/imdb", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]
dataset["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 20000
})

In [15]:
dataset["train"][0]

{'text': 'The stories in this video are very entertaining, and it definately is worth a look! The first one concerns a young couple harrassed in the woods by two rednecks, with a great, but unexplained twist at the end.<br /><br />The seond is the best of the lot, and it alone, makes this worth watching - A man is attacked by a dog, which he fears to be rabid - He finds shelter in what appears to be a hospital, but he finds out the employees there are not exactly what they appear to be...... Great twist at the end, and this episode alone scores 10/10! If the others were up to par with this one, this would get 10/10!<br /><br />The third is the weakest of the bunch - A girl meets with some guys and has wild sex! There appears to be no point to the story until the end, with a good little twist, but it is spoiled by the awful first part!<br /><br />Never the less, this is a great movie that will not do you wrong at all! Well worth a rental!',
 'label': 1}

### Loading the Tokenizer and tokenize the dataset

In [16]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], truncation=True), batched=True
    )

tokenized_dataset["train"]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 20000
})

### Loading the pretrained HuggingFace foundation model

In [17]:
model = AutoModelForSequenceClassification.from_pretrained(
    'gpt2', 
    num_labels=2,
    id2label={0: 'NEGATIVE', 1: 'POSITIVE'},
    label2id={'NEGATIVE': 0, 'POSITIVE': 1}
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
for param in model.parameters():
    param.requires_grad = True

In [19]:
print(model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)


### Evaluating the model

In [20]:
model.score

Linear(in_features=768, out_features=2, bias=False)

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.